# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [15]:
# import libraries
import re
import pandas as pd
import nltk
import pickle
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [16]:
nltk.download(['punkt', 'wordnet', 'omw-1.4'])

[nltk_data] Downloading package punkt to /home/pqt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/pqt/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/pqt/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [17]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponseTable', engine)
X = df.message
# load features set
y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [18]:
def tokenize(text):
    # url regex for matching an URL
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    found_urls = re.findall(url_regex, text)
    for url in found_urls:
        text = text.replace(url, 'url')
    # tokenize words, normalize and remove empty spaces
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    processed_tokens = [lemmatizer.lemmatize(token).lower().strip() for token in tokens]
    return processed_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [19]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fe36cf00040>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:
y_pred = pipeline.predict(X_test)
i = 0
for col in y_test:
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))
    i += 1
print('Accuracy: {:.3f}'.format((y_pred == y_test.values).mean()))

('related',)
              precision    recall  f1-score   support

           0       0.72      0.28      0.41      1475
           1       0.81      0.97      0.88      5012
           2       0.88      0.10      0.19        67

    accuracy                           0.81      6554
   macro avg       0.80      0.45      0.49      6554
weighted avg       0.79      0.81      0.77      6554

('request',)
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5403
           1       0.89      0.42      0.57      1151

    accuracy                           0.89      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.89      0.87      6554

('offer',)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6532
           1       0.00      0.00      0.00        22

    accuracy                           1.00      6554
   macro avg       0.50      0.50    

/mnt/d/SAS/Stuffs/udacity_nd025/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/d/SAS/Stuffs/udacity_nd025/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/d/SAS/Stuffs/udacity_nd025/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6484
           1       0.00      0.00      0.00        70

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

('shops',)
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6516
           1       0.00      0.00      0.00        38

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

('aid_centers',)
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6470
           1       0.00      0.00      0.00        84

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.97      0.99      0.98   

/mnt/d/SAS/Stuffs/udacity_nd025/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/d/SAS/Stuffs/udacity_nd025/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
# Show parameters for the pipline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7fe36cf00040>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7fe36cf00040>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [23]:
parameters = {
    #'vect__ngram_range': ((1, 1),(1,2)),
    #'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000),
    # 'tfidf__use_idf': (True, False),
    # 'clf__estimator__n_estimators': [50],
    'clf__estimator__n_estimators': [10],
    'clf__estimator__min_samples_split': [2],
    # 'clf__estimator__min_samples_split': [2, 3, 4],
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=2)
cv.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10; total time=  47.3s
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10; total time=  46.7s
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10; total time=  47.6s
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10; total time=  47.1s
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10; total time=  24.2s


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fe36cf00040>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=4,
             param_grid={'clf__estimator__min_samples_split': [2],
                         'clf__estimator__n_estimators': [10]},
             verbose=2)

In [24]:
cv.cv_results_

{'mean_fit_time': array([40.06555824]),
 'std_fit_time': array([8.72938361]),
 'mean_score_time': array([2.53507376]),
 'std_score_time': array([0.456802]),
 'param_clf__estimator__min_samples_split': masked_array(data=[2],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_clf__estimator__n_estimators': masked_array(data=[10],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__estimator__min_samples_split': 2,
   'clf__estimator__n_estimators': 10}],
 'split0_test_score': array([0.21103483]),
 'split1_test_score': array([0.2100178]),
 'split2_test_score': array([0.22558494]),
 'split3_test_score': array([0.21998983]),
 'split4_test_score': array([0.22863683]),
 'mean_test_score': array([0.21905285]),
 'std_test_score': array([0.00750093]),
 'rank_test_score': array([1], dtype=int32)}

In [25]:
cv.best_params_

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 10}

In [26]:
optimised_model = cv.best_estimator_
cv.best_estimator_

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fe36cf00040>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10)))])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
y_pred = optimised_model.predict(X_test)
i = 0
for col in y_test:
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))
    i += 1
print('Accuracy: {:.3f}'.format((y_pred == y_test.values).mean()))

('related',)
              precision    recall  f1-score   support

           0       0.60      0.35      0.44      1475
           1       0.82      0.93      0.87      5012
           2       0.67      0.12      0.20        67

    accuracy                           0.79      6554
   macro avg       0.70      0.47      0.51      6554
weighted avg       0.77      0.79      0.77      6554

('request',)
              precision    recall  f1-score   support

           0       0.88      0.98      0.93      5403
           1       0.81      0.35      0.49      1151

    accuracy                           0.87      6554
   macro avg       0.84      0.66      0.71      6554
weighted avg       0.86      0.87      0.85      6554

('offer',)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6532
           1       0.00      0.00      0.00        22

    accuracy                           1.00      6554
   macro avg       0.50      0.50    

/mnt/d/SAS/Stuffs/udacity_nd025/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/d/SAS/Stuffs/udacity_nd025/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/d/SAS/Stuffs/udacity_nd025/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      6036
           1       0.84      0.34      0.48       518

    accuracy                           0.94      6554
   macro avg       0.90      0.67      0.73      6554
weighted avg       0.94      0.94      0.93      6554

('storm',)
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      5947
           1       0.74      0.30      0.43       607

    accuracy                           0.93      6554
   macro avg       0.84      0.65      0.70      6554
weighted avg       0.92      0.93      0.91      6554

('fire',)
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6484
           1       0.00      0.00      0.00        70

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [28]:
with open('classifier.pkl', 'wb') as file:
    pickle.dump(optimised_model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.